# Install all Required Packages

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curre

# Import all Required Libraries

In [ ]:
import os
import torch
from datasets import load_dataset, DatasetDict, Dataset, concatenate_datasets
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoModelForQuestionAnswering,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Loading Dataset

In [ ]:
data1= load_dataset("ShenLab/MentalChat16K")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

Interview_Data_6K.csv:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

Synthetic_Data_10K.csv:   0%|          | 0.00/32.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16084 [00:00<?, ? examples/s]

# Task: Question-Answering

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "google/flan-t5-base"


# Fine-tuned model name
new_model2 = "google/flan-t5-base-QnA-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
data1

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 16084
    })
})

In [ ]:
from datasets import load_dataset, DatasetDict

# Load the dataset
dataset = load_dataset("ShenLab/MentalChat16K")

# Split the dataset into train and test
train_test_split = dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']


train_valid_split = train_dataset.train_test_split(test_size=0.1)
train_dataset = train_valid_split["train"]
validation_dataset = train_valid_split["test"]

# Combine into a DatasetDict
split_dataset = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
})

print(split_dataset)
dataset = split_dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

Interview_Data_6K.csv:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

Synthetic_Data_10K.csv:   0%|          | 0.00/32.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16084 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 11580
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1287
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 3217
    })
})


In [ ]:
from transformers import AutoTokenizer
from datasets import DatasetDict


# Initialize the tokenizer for Flan-T5
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")


# preprocessing function
def preprocess_qa(examples):
    prompts = [
        f"{inst} {context}" if context else inst
        for inst, context in zip(examples["instruction"], examples["input"])
    ]

    # Tokenize inputs for the model
    model_inputs = tokenizer(prompts, max_length=512, truncation=True, padding="max_length")

    # Tokenize answers (targets) and set as labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["output"], max_length=128, truncation=True, padding="max_length").input_ids

    model_inputs["labels"] = labels  # Add tokenized answers as labels
    return model_inputs

    dataset= dataset.map(preprocess_qa, batched= True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
# Preprocess each split
train_dataset = train_dataset.map(preprocess_qa, batched=True)
validation_dataset = validation_dataset.map(preprocess_qa, batched=True)
test_dataset = test_dataset.map(preprocess_qa, batched=True)

# Print to confirm processed datasets
print("Processed Train Dataset:", train_dataset)
print("Processed Validation Dataset:", validation_dataset)
print("Processed Test Dataset:", test_dataset)


Map:   0%|          | 0/11580 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

Map:   0%|          | 0/3217 [00:00<?, ? examples/s]

Processed Train Dataset: Dataset({
    features: ['instruction', 'input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 11580
})
Processed Validation Dataset: Dataset({
    features: ['instruction', 'input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1287
})
Processed Test Dataset: Dataset({
    features: ['instruction', 'input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3217
})


In [ ]:
# Define BitsAndBytesConfig if you are using 4-bit quantization (skip if not needed)
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load the Flan-T5 model
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="SEQ_2_SEQ_LM"  # Compatible with Flan-T5's Seq2Seq setup
)

# Set training parameters

training_arguments = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    predict_with_generate=True,
    report_to="tensorboard"
)

# Initialize Seq2SeqTrainer without any additional formatting
trainer = Seq2SeqTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,2.958100
50,3.753500
75,2.606700
100,2.624000
125,2.429300
150,2.918800
175,2.463000
200,2.780400
225,2.502100
250,2.221400


TrainOutput(global_step=2895, training_loss=1.8690343669041452, metrics={'train_runtime': 956.3275, 'train_samples_per_second': 12.109, 'train_steps_per_second': 3.027, 'total_flos': 5075903132467200.0, 'train_loss': 1.8690343669041452, 'epoch': 1.0})

In [ ]:
# Load the base model in standard precision
base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", device_map="auto")
model = PeftModel.from_pretrained(base_model, "DiatWork/flan-t5-MentalHealth-fine-tuned")




Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user v

In [ ]:
# Model save in directory
save_directory = "./fine-tuned-flan-t5-mental-health"

# Save the model and adapter weights
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
tokenizer.save_pretrained(save_directory)


('./fine-tuned-flan-t5-mental-health/tokenizer_config.json',
 './fine-tuned-flan-t5-mental-health/special_tokens_map.json',
 './fine-tuned-flan-t5-mental-health/spiece.model',
 './fine-tuned-flan-t5-mental-health/added_tokens.json',
 './fine-tuned-flan-t5-mental-health/tokenizer.json')

# Saving Model to Drive

In [ ]:
new_model= "google/flan-t5-base-QnA-finetune"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
output_dir= "/content/drive/MyDrive/flan-T5-chat-finetuned"


In [ ]:
from peft import PeftModel

# Ensure the model is wrapped as a PeftModel
peft_model = PeftModel(model, peft_config)

# Define the output directory path (can be in Google Drive or any local directory)
adapter_output_dir = "/content/drive/MyDrive/flan-T5-chat-adapter"

# Save the LoRA adapters
peft_model.save_pretrained(adapter_output_dir)
tokenizer.save_pretrained(adapter_output_dir)  # Save tokenizer config as usual

('/content/drive/MyDrive/flan-T5-chat-adapter/tokenizer_config.json',
 '/content/drive/MyDrive/flan-T5-chat-adapter/special_tokens_map.json',
 '/content/drive/MyDrive/flan-T5-chat-adapter/spiece.model',
 '/content/drive/MyDrive/flan-T5-chat-adapter/added_tokens.json',
 '/content/drive/MyDrive/flan-T5-chat-adapter/tokenizer.json')

# Loading model from Drive

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig

# Load the base model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the adapter
peft_config = PeftConfig.from_pretrained(adapter_output_dir)
model = PeftModel.from_pretrained(model, adapter_output_dir)

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user v

In [ ]:
import os

adapter_output_dir = "/content/drive/MyDrive/flan-T5-chat-adapter"
print("Saved files:", os.listdir(adapter_output_dir))


Saved files: ['README.md', 'adapter_model.bin', 'adapter_config.json', 'tokenizer_config.json', 'special_tokens_map.json', 'spiece.model', 'tokenizer.json']


# Save Model in Hugging Face Repo

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import create_repo

repo_name = "DiatWork/flan-t5-MentalHealth-Chat-fine-tuned"
create_repo(repo_name, exist_ok=True)


RepoUrl('https://huggingface.co/DiatWork/flan-t5-MentalHealth-Chat-fine-tuned', endpoint='https://huggingface.co', repo_type='model', repo_id='DiatWork/flan-t5-MentalHealth-Chat-fine-tuned')

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/flan-T5-chat-adapter")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")


In [ ]:
model.push_to_hub(repo_id= repo_name)
tokenizer.push_to_hub(repo_id= repo_name)



adapter_model.bin:   0%|          | 0.00/28.4M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DiatWork/flan-t5-MentalHealth-Chat-fine-tuned/commit/9925a5d44fa05fa64c5e083e8320d5d6f2b20812', commit_message='Upload tokenizer', commit_description='', oid='9925a5d44fa05fa64c5e083e8320d5d6f2b20812', pr_url=None, pr_revision=None, pr_num=None)

# Load Model from HuggingFace


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel

# Load the full base model first
base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", device_map="auto")

# Load the LoRA adapter on top of the fully loaded base model
model = PeftModel.from_pretrained(base_model, "DiatWork/flan-t5-MentalHealth-fine-tuned")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", force_download=True)


Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/28.4M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

In [ ]:
dataset['test']

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 3217
})

In [ ]:
from transformers import GenerationConfig

index = 100
context = dataset['test'][index]['instruction']
question = dataset['test'][index]['input']
human_baseline_answer = dataset['test'][index]['output']

# Define the question-answering prompt
prompt = f"""
Use the context below to answer the question.

Context: {context}

Question: {question}

Answer:
"""

# Tokenize and move `input_ids` to the same device as the model
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

# Generate answers from the model
generation_config = GenerationConfig(
    max_new_tokens=100,
    num_beams=3,            # More beams may improve quality
    repetition_penalty=1.2, # Reduces repetitive responses
    temperature=0.7,        # Controls randomness; lower for focused answers
    top_p=0.9               # Nucleus sampling for more diverse output
)
instruct_model_outputs = model.generate(input_ids=input_ids, generation_config=generation_config)

# Decode the output
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

# Display results
dash_line = "-" * 50
print(dash_line)
print(f'HUMAN BASELINE ANSWER:\n{human_baseline_answer}')
print(dash_line)
print(f'INSTRUCT MODEL ANSWER:\n{instruct_model_text_output}')


--------------------------------------------------
HUMAN BASELINE ANSWER:
There are several therapy options and techniques that have been found to be effective in treating depression. One common approach is Cognitive Behavioral Therapy (CBT), which focuses on identifying negative thought patterns and replacing them with more positive and adaptive ones. CBT can help you develop coping strategies and skills to manage your symptoms.

Another option is Interpersonal Therapy (IPT), which explores how your relationships and social interactions may contribute to your depression. IPT aims to improve communication, problem-solving, and relationship building skills, which can positively impact your mood.

Additionally, Psychodynamic Therapy delves into the unconscious roots of your depression, aiming to uncover unresolved conflicts or past traumas that may be contributing to your current struggles.

Mindfulness-based therapies, such as Mindfulness-Based Cognitive Therapy (MBCT) or Acceptance and

# Inference Test

In [ ]:
# Define the input text
input_text = (
    "You are a compassionate mental health counseling assistant trained to support caregivers of Alzheimer's patients. "
    "Please respond to the following question with empathy, understanding, and actionable advice. "
    "My husband's recent diagnosis with a terminal illness has left me feeling helpless and overwhelmed. I've always defined success as achieving career milestones, but now, with his illness, I can't help but feel that my own success is meaningless compared to his suffering."
)

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Generate a response with adjusted parameters
output = model.generate(
    **inputs,
    max_length=300,
    num_beams=5,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.2  # Helps reduce repetition
)

# Decode the generated response
response = tokenizer.decode(output[0], skip_special_tokens=True)
print("Adjusted Model Response:", response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Adjusted Model Response: I've always defined success as achieving career milestones, but now, with his illness, I can't help but feel that my own success is meaningless compared to his suffering. I've always defined success as achieving career milestones, but now, with his illness, I can't help but feel that my own success is meaningless compared to his suffering.


In [ ]:
!pip install rouge_score evaluate bert_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=2f6a2d795f15650a720c36d79d47d628572e28ef021fef1e23301adfb62bd648
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import textwrap
import torch
import evaluate

test_data = dataset["test"]

prompts = []
references = []

# Build prompts and references from the validation dataset
for i in range(int(0.5*len(test_data))):
    sample = test_data[i]
    instruction = sample['instruction']
    input_text = sample['input']
    output_text = sample['output']

    if input_text:
        prompt = f"<s>[INST] {instruction} {input_text} [/INST]"
    else:
        prompt = f"<s>[INST] {instruction} [/INST]"

    prompts.append(prompt)
    references.append(output_text)  # reference answer

# Generate responses for each prompt and format them
generated_texts = []
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            top_p=0.9,
            top_k=50,
            temperature=0.7,
            use_cache=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the response
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Split question and model answer
    try:
        question, model_answer = decoded_output.split("[/INST]", 1)
    except ValueError:
        question, model_answer = decoded_output, ""

    question = "\n".join(textwrap.wrap(question, width=80))
    model_answer = "\n".join(textwrap.wrap(model_answer, width=80))

    print("Question:\n", question)
    print("\nModel Answer:\n", model_answer)
    print("\n" + "="*80 + "\n")  # Separator for readability

    # Append the raw decoded output to generated_texts for evaluation
    generated_texts.append(decoded_output)



!pip install rouge_score

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

#Calculate BLEU, ROUGE, and BERTScore
bleu_score = bleu.compute(predictions=generated_texts, references=[[ref] for ref in references])
rouge_score = rouge.compute(predictions=generated_texts, references=references)
bert_score = bertscore.compute(predictions=generated_texts, references=references, lang="en")

# Print Evaluation Results
print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)
print("BERTScore (Precision, Recall, F1):", bert_score["precision"], bert_score["recall"], bert_score["f1"])

from huggingface_hub import Repository

repo = Repository(local_dir="./flanT5_eval_score", clone_from="DiatWork/flan-t5-MentalHealth-fine-tuned")

# Save metrics to JSON file
evaluation_metrics = {
    "ROUGE": rouge_score,
    "BERTScore": bert_score,
    "BLEU": bleu_score,
}




Question:
 I want counseling to help me understand the root causes of my anxiety and learn
effective techniques to calm myself during episodes of heightened anxiety

Model Answer:
 


Question:
 s>[INST]

Model Answer:
 


Question:
 Have you seen any success in treating depression through therapy alone? Or do
most patients require medication as well?

Model Answer:
 


Question:
 I've been wrestling with feelings of inadequacy and self-doubt that have been
plaguing me for years.

Model Answer:
 




KeyboardInterrupt: 

# stopped the evaluation since the model was not generating any output and performed bad at Inference